IMPORTS

In [1]:
import numpy as np
import cv2
import os

FUNCTION TO DISPLAY AN IMAGE

In [2]:
# Function from lab2
def show_image(image, window_name='image', timeout=0):
    cv2.imshow(window_name, cv2.resize(image, None, fx=0.6, fy=0.6))
    cv2.waitKey(timeout)
    cv2.destroyAllWindows()

FUNCTION TO READ THE IMAGES FROM FOLDER

In [3]:
#Read images from dataset
def read_images():
    img = []
    folder_path = 'Task2/'
    
    for i in range(1, 26):
        filename = '0' + str(i) if i < 10 else str(i)
        img_path = os.path.join(folder_path, filename + '.jpg')
        img.append(cv2.imread(img_path))
    
    return img

FUNCTION TO DETECT THE NUMBER OF DARTS ON THE DARTBOARD

A threshold was applied so that we can detect the pixels of interest, we isolate the colored pixels seeing that the image is predominantly black and white with darts colored.

In order to find the number of darts on the dartboard I firstly computed the absolute difference between the template photo and the photo with darts on it, so I could remain only with the darts. After that I applied a threshold so that I could remain only with the colored parts, after that I converted the image to hsv format so we can precess it without being influenced by changes in lighting conditions. The next step was to apply a red mask and a green mask to detect the flags, I've applied img_erosion method from cv2 to get rid of extra lines from the dartboard and after that dilate method to recognise correctly the number of flags, I've selected the parameters for those by testing how it works on all images.

At the end, I've selected only poligons with width and height bigger than 90 (arbitrary)

In [4]:
def detectDarts(image,template):
    # Compute the absolute difference between the template and our image
    diff = 255 - cv2.absdiff(image, template)

    tmp = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # This was used because I saw an improvement in the general detection
    # If the pixel value is smaller than the threshold, it is set to 0, otherwise it is set to a maximum value.
    _,alpha = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY)
    
    # Split channels our image 
    b, g, r = cv2.split(diff)
    rgba = [b, g, r, alpha]
    dst = cv2.merge(rgba, 4)
    
    # Transform the picture to hsv
    img_hsv=cv2.cvtColor(dst, cv2.COLOR_BGR2HSV)

    # Apply a red mask for the first red part in hsv map
    lower_red = np.array([0,50,50])
    upper_red = np.array([10,255,255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

    # Mask for the other red part on hsv map (170-180)
    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(img_hsv, lower_red, upper_red)

    # Apply green mask
    lower_green = np.array([35, 50, 50])  
    upper_green = np.array([85, 255, 255])
    green_mask = cv2.inRange(img_hsv, lower_green, upper_green)

    # Join all the masks
    mask = mask0 + mask1 + green_mask
    
    # Define kernels with values chosen manually by testing
    kernel = np.ones((100, 100), np.uint8)
    kernel2 = np.ones((10, 10), np.uint8)

    # Apply erosion so we can get rid of the fine lines
    img_erosion = cv2.erode(mask, kernel2, iterations=1)

    # Dilate the points representing darts so that the detection works better
    img_dilation = cv2.dilate(img_erosion, kernel, iterations=1)

    # Find contours and count them
    edges = cv2.Canny(img_dilation, 50, 150)
    contours, hierarchy = cv2.findContours(img_dilation,
                                           cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(image=img_dilation, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

    # Remove small contours
    darts_num = 0
    darts = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if w > 90 and h > 90:
            darts.append(c)
            darts_num += 1
            
    return darts, darts_num

I tried two different approaches in order to detect the ares of interest in the pictures but none of them seemed to give satisfactory results. Lastly I selected the areas manually.

So the following two methods are not used

In [5]:
# ! Not used in solution
def find_circles(image):

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    lower_green = np.array([35, 50, 50]) 
    upper_green = np.array([85, 255, 255])
    green_mask = cv2.inRange(hsv, lower_green, upper_green)

    copy_image = image
    copy_image[green_mask>0]=(255,255,255)

    hsv = cv2.cvtColor(copy_image, cv2.COLOR_BGR2HSV)
    
    # Apply a red mask for the first red part in hsv map
    lower_red = np.array([0,50,50])
    upper_red = np.array([10,255,255])
    mask0 = cv2.inRange(hsv, lower_red, upper_red)
    
    # Mask for the other red part on hsv map (170-180)
    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    # Join my masks
    mask = mask0 + mask1

    inversed_mask = cv2.bitwise_not(mask)
    Mask = cv2.bitwise_or(copy_image, copy_image, mask = inversed_mask)

    # Filter image
    thresh = 128
    img_binary = cv2.threshold(Mask, thresh, 255, cv2.THRESH_BINARY)[1]
    
    # Apply erosion so we can get rid of the fine lines
    kernel = np.ones((5, 5), np.uint8)
    img_erosion = cv2.erode(img_binary, kernel, iterations=1)
  
    contours, hierarchy = cv2.findContours(image=img_binary, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(image=img_binary, contours=contours, contourIdx=-1, color=(0, 0, 255), thickness=13, lineType=cv2.LINE_AA)
    
    return contours

Callback function to capture mouse events was used in order to generate the points for the 20 polygons representing regions on the dartboard

In [6]:

def get_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click event
        print(f"Selected point coordinates: ({x}, {y})")

def get_points():
    image = cv2.imread('auxiliary_images/template_task2.jpg')
    cv2.imshow('Image', image)
    cv2.setMouseCallback('Image', get_coordinates)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

The 20 polygons representing regions on the dartboard manually defined

In [7]:
p12 = np.array([[778, 1250], [965, 1087],
                [1284, 1790]
                ],
               np.int32)

p9 = np.array([[628, 1495], [772, 1259],
                [1272, 1804]
                ],
               np.int32)

p14 = np.array([[568, 1790], [636, 1508],
                [1265, 1828]
                ],
               np.int32)

p11 = np.array([[586, 2101], [568, 1800],
                [1265, 1852]
                ],
               np.int32)

p8 = np.array([[685, 2372], [588, 2106],
                [1262, 1892]
                ],
               np.int32)

p16 = np.array([[851, 2584], [690, 2377],
                [690, 2377], [1284, 1915]
                ],
               np.int32)

p7 = np.array([[1055, 2687], [854, 2591],
                [854, 2591]
                ],
               np.int32)

p19 = np.array([[1272, 2715], [1061, 2690],
                [1298, 1925], [1322, 1931]
                ],
               np.int32)

p3 = np.array([[1474, 2653], [1283, 2718],
                [1330, 1944]
                ],
               np.int32)

p17 = np.array([[1642, 2515], [1477, 2654],
                [1343, 1940], [1357, 1934]
                ],
               np.int32)

p2 = np.array([[1776, 2336], [1655, 2518],
                [1375, 1919]
                ],
               np.int32)

p15 = np.array([[1863, 2139], [1785, 2325],
                [1379, 1912], [1392, 1887]
                ],
               np.int32)

p10 = np.array([[1901, 1900], [1865, 2128],
                [1398, 1875]
                ],
               np.int32)

p6 = np.array([[1888, 1657], [1898, 1887],
                [1396, 1854], [1396, 1832]
                ],
               np.int32)

p5 = np.array([[966, 1092], [1176, 1013],
                [1317, 1765], [1297, 1775]
                ],
               np.int32)

p20 = np.array([[1174, 1007], [1380, 1019],
                [1318, 1769]
                ],
               np.int32)

p1 = np.array([[1388, 1022], [1567, 1112],
                [1359, 1768], [1335, 1768]
                ],
               np.int32)

p18 = np.array([[1572, 1113], [1715, 1255],
                [1362, 1777]
                ],
               np.int32)

p4 = np.array([[1722, 1260], [1831, 1442],
                [1388, 1801], [1365, 1780]
                ],
               np.int32)

p13 = np.array([[1833, 1447], [1885, 1663],
                [1394, 1811]
                ],
               np.int32)


Arrays with the 20 polygons + its etiquettes were defined manually

In [8]:
polygons = [p12, p9, p14, p11, p8, p16, p7, p19, p3, p17, p2, p15, p10, p6, p5, p20, p1, p18, p4, p13]
etiquette = [12, 9, 14, 11, 8, 16, 7, 19, 3, 17, 2, 15, 10, 6, 5, 20, 1, 18, 4, 13]

FUNCTION TO DETECT THE POINT REPRESENTING THE TIP OF THE DART

Similar to approach in Task 1

In order to find this point I computed first the centre of the poligon representing the flag, and after analysing all the pictures in the train dataset I observed that most of the darts tips are located around 450 units to the left and 10 units to the bottom relative to the centre of the flag, so I used this euristic method in order to determine the tip.

In [9]:
def detectTipPosition(darts, img):
    # Inspiration for the blob center computation:
    # https://learnopencv.com/find-center-of-blob-centroid-using-opencv-cpp-python/
    tips_list = []
    for dart in darts:

        dart_center = cv2.moments(dart)
        # Computing the centre for each dart using moments method from cv2
        cX = int(dart_center["m10"] / dart_center["m00"])
        cY = int(dart_center["m01"] / dart_center["m00"])

        # Estimate the tip
        cA = int(cX - 450)
        cB = int(cY + 10)
        tips_list.append((cA, cB))
        
        # # Code for drawing a marker on tip (testing purposes)
        # color = (0, 255, 0)
        # markerType = cv2.MARKER_CROSS
        # markerSize = 50
        # thickness = 5
        # cv2.drawMarker(img, (cA, cB), color, markerType, markerSize, thickness)
    # show_image(img, "test")
    return tips_list

FUNCTION TO WRITE THE PREDICTION IN FILE

In [10]:
def writeInFile(darts_num, scores_list, i):

    file_identifier = i + 1
    filename = ('0' + str(file_identifier) if file_identifier < 10 else str(file_identifier))
    path = 'Ilicea_Anca_512/Task2/' + filename + '_predicted.txt'

    print(path)
    f = open(path, 'w')
    f.write(str(darts_num) + '\n')

    for score in scores_list:
        f.write(str(score) + '\n')

THIS FUNCTION WILL ESTIMATE THE SUBREGION IN WHICH A TIP OF A DART CAN BE FOUND

For each dart tip coordate we will iterate over through the poligons list and check if it is contained in it, if not we will set a fixed value 's20' ( none of the poligons overlap with each other). There were considered only subregions.

In [11]:
def estimateScore(countours, tips_list):
    scores_list = []

    for dart in tips_list:
        for j, poly in enumerate(countours):
            found = 0 
            if cv2.pointPolygonTest(poly, dart, False) >= 0:
                found = 1
                scores_list.append('s' + str(etiquette[j]))
                break
        if(found == 0):
            scores_list.append('s20')
    return scores_list

MAIN FUNCTION, IT WILL RUN ALL THE FUNCTIONS AND GENERATE THE SOLUTION

In [12]:
def main():
    # Get the template so we can calculate the elipsis
    template = cv2.imread('auxiliary_images/template_task2.jpg')
    # Get all the train images
    img_array = read_images()
    # Iterate through all images
    for i, img in enumerate(img_array):
        # Detect the number of darts
        darts, darts_num = detectDarts(img_array[i], template)
        # Get the tip point for each dart
        tips_list = detectTipPosition(darts, img_array[i])
        # Estimate the regions in which the dart tips are located
        scores_list = estimateScore(polygons, tips_list)
        # Write predictions in file 
        writeInFile(darts_num, scores_list, i)
main()

Ilicea_Anca_512/Task2/01_predicted.txt
Ilicea_Anca_512/Task2/02_predicted.txt
Ilicea_Anca_512/Task2/03_predicted.txt
Ilicea_Anca_512/Task2/04_predicted.txt
Ilicea_Anca_512/Task2/05_predicted.txt
Ilicea_Anca_512/Task2/06_predicted.txt
Ilicea_Anca_512/Task2/07_predicted.txt
Ilicea_Anca_512/Task2/08_predicted.txt
Ilicea_Anca_512/Task2/09_predicted.txt
Ilicea_Anca_512/Task2/10_predicted.txt
Ilicea_Anca_512/Task2/11_predicted.txt
Ilicea_Anca_512/Task2/12_predicted.txt
Ilicea_Anca_512/Task2/13_predicted.txt
Ilicea_Anca_512/Task2/14_predicted.txt
Ilicea_Anca_512/Task2/15_predicted.txt
Ilicea_Anca_512/Task2/16_predicted.txt
Ilicea_Anca_512/Task2/17_predicted.txt
Ilicea_Anca_512/Task2/18_predicted.txt
Ilicea_Anca_512/Task2/19_predicted.txt
Ilicea_Anca_512/Task2/20_predicted.txt
Ilicea_Anca_512/Task2/21_predicted.txt
Ilicea_Anca_512/Task2/22_predicted.txt
Ilicea_Anca_512/Task2/23_predicted.txt
Ilicea_Anca_512/Task2/24_predicted.txt
Ilicea_Anca_512/Task2/25_predicted.txt
